In [1]:
import wandb

In [2]:
wandb.login()

In [3]:
import pandas as pd
import os
import tqdm
import numpy as np
import json
import tensorflow as tf
import gc
from os import mkdir

import math
import pickle
import matplotlib.pyplot as plt
from keras import layers, models
import tensorflow_addons as tfa

In [5]:
with open (r"C:\Users\chuqi\ac215\kaggle-data\character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)
    
pad_token = '^'
pad_token_idx = 59

char_to_num[pad_token] = pad_token_idx
num_to_char = {j:i for i,j in char_to_num.items()}

In [6]:
df = pd.read_csv(r"how2sign_realigned_test.csv",index_col=0)
df['FILE_ID'] = df['SENTENCE_NAME'].apply(lambda x: "preprocessed_"+x+'.npy') 
df.head()

In [7]:
# read capy csv 
df_pre = pd.read_csv(r"C:/Users/chuqi/ac215/capy-data/test/preprocessed__fZbAxSSbX4_0-5-rgb_front.csv",index_col=0)
cols = df_pre.columns.tolist()[2:]
cols[:5]

In [8]:
FACE = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]

LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

In [9]:
# FACE = [int(cor.split("_")[-1]) for cor in cols if 'x_face' in cor]
HAND = [int(cor.split("_")[-1]) for cor in cols if 'x_right_hand' in cor]
# POSE = [int(cor.split("_")[-1]) for cor in cols if 'x_pose' in cor]
# LPOSE = [i for i in POSE if i%2!=0]
# RPOSE = [i for i in POSE if i%2==0]

LPOSE_cols_x = ['x'+'_pose_'+str(i) for i in LPOSE]
LPOSE_cols_y = ['y'+'_pose_'+str(i) for i in LPOSE]
LPOSE_cols_z = ['z'+'_pose_'+str(i) for i in LPOSE]

RPOSE_cols_x = ['x'+'_pose_'+str(i) for i in RPOSE]
RPOSE_cols_y = ['y'+'_pose_'+str(i) for i in RPOSE]
RPOSE_cols_z = ['z'+'_pose_'+str(i) for i in RPOSE]

In [10]:
# FACE
FACE_IDX_X = [i for i, col in enumerate(cols) if 'face' in col and "x" in col and int(col[7:]) in FACE]
FACE_IDX_Y = [i for i, col in enumerate(cols) if 'face' in col and "y" in col and int(col[7:]) in FACE]
FACE_IDX_Z = [i for i, col in enumerate(cols) if 'face' in col and "z" in col and int(col[7:]) in FACE]

# HAND 
RHAND_IDX_X = [i for i, col in enumerate(cols) if 'x_right_hand' in col]
LHAND_IDX_X = [i for i, col in enumerate(cols) if 'x_left_hand' in col]

RHAND_IDX_Y = [i for i, col in enumerate(cols) if 'y_right_hand' in col]
LHAND_IDX_Y = [i for i, col in enumerate(cols) if 'y_left_hand' in col]

RHAND_IDX_Z = [i for i, col in enumerate(cols) if 'z_right_hand' in col]
LHAND_IDX_Z = [i for i, col in enumerate(cols) if 'z_left_hand' in col]

# POSE RIGHT
RPOSE_IDX_X = [i for i, col in enumerate(cols) if col in RPOSE_cols_x]
RPOSE_IDX_Y = [i for i, col in enumerate(cols) if col in RPOSE_cols_y]
RPOSE_IDX_Z = [i for i, col in enumerate(cols) if col in RPOSE_cols_z]

# POSE LEFT
LPOSE_IDX_X = [i for i, col in enumerate(cols) if col in LPOSE_cols_x]
LPOSE_IDX_Y = [i for i, col in enumerate(cols) if col in LPOSE_cols_y]
LPOSE_IDX_Z = [i for i, col in enumerate(cols) if col in LPOSE_cols_z]

In [11]:
def load_npy_data(file_id):
    folder = "C:/Users/chuqi/ac215/capy-data/test_npy"
    file_name = os.path.join(folder,file_id)
    # Delete the first 3 columns: index, file_name, frame num
    return np.load(file_name,allow_pickle=True)[:,3:]

def fmkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

folder ="C:/Users/chuqi/ac215/capy_data_test/test_npy"
file_ls = os.listdir(folder)
for file_id in tqdm.tqdm(file_ls):
    frames = load_npy_data(file_id)
    phrase = str(df[df['FILE_ID']==file_id]['SENTENCE'].iloc[0])
    
    def process(x):
        face_x = x[:, FACE_IDX_X]
        face_y = x[:, FACE_IDX_Y]
        face_z = x[:, FACE_IDX_Z]

        rhand_x = x[:, RHAND_IDX_X]
        rhand_y = x[:, RHAND_IDX_Y]
        rhand_z = x[:, RHAND_IDX_Z]
        
        lhand_x = x[:, LHAND_IDX_X]
        lhand_y = x[:, LHAND_IDX_Y]
        lhand_z = x[:, LHAND_IDX_Z]

        rpose_x = x[:, RPOSE_IDX_X]
        rpose_y = x[:, RPOSE_IDX_Y]
        rpose_z = x[:, RPOSE_IDX_Z]
        
        lpose_x = x[:, LPOSE_IDX_X]
        lpose_y = x[:, LPOSE_IDX_Y]
        lpose_z = x[:, LPOSE_IDX_Z]
                
        rhnonans = ~np.isnan(np.sum(rhand_x, axis=1).astype(float))
        lhnonans = ~np.isnan(np.sum(lhand_x, axis=1).astype(float))
        rphonans = ~np.isnan(np.sum(rpose_x, axis=1).astype(float))
        lphonans = ~np.isnan(np.sum(lpose_x, axis=1).astype(float))
        fcnonans = ~np.isnan(np.sum(face_x,   axis=1).astype(float))
        
        rhand = np.stack([rhand_x, rhand_y, rhand_z], axis=-1)[rhnonans]
        rpose = np.stack([rpose_x, rpose_y, rpose_z], axis=-1)[rphonans]
        lhand = np.stack([lhand_x, lhand_y, lhand_z], axis=-1)[lhnonans]
        lpose = np.stack([lpose_x, lpose_y, lpose_z], axis=-1)[lphonans]
        face = np.stack([face_x, face_y, face_z], axis=-1)[fcnonans]

        return rhand, lhand, rpose, lpose, face

    rhand, lhand, rpose, lpose, face = process(frames)
    print(rhand.shape, lhand.shape, rpose.shape, lpose.shape, face.shape)
    
    def gen(df):
        # extract numpy file 
        folder_directory = r"C:/Users/chuqi/ac215/capy_data_test/test_npy"
        file_ls = [f for f in os.listdir(folder_directory) if '.npy' in f]
        for file_id in file_ls:
            try:
                # pqfile = f"/kaggle/input/asl-fingerspelling/train_landmarks/{file_id}.parquet"
                npy_file = f"C:/Users/chuqi/ac215/capy-data/test_npy/{file_id}"
                frames = load_npy_data(file_id)
                phrase = df[df['FILE_ID']==file_id]["SENTENCE"].iloc[0]
                
                # Preprocess
                rhand, lhand, rpose, lpose, face = process(frames)
                if max(rhand.shape[0], lhand.shape[0]) > len(phrase):
                    yield rhand, lhand, rpose, lpose, face
            except:
                print("Error file:", file_id)
    RHAND = []
    LHAND = []
    RPOSE = []
    LPOSE = []
    FACE = []

    for rhand, lhand, rpose, lpose, face in gen(df):
        RHAND.extend(rhand)
        LHAND.extend(lhand)
        RPOSE.extend(rpose)
        LPOSE.extend(lpose)
        FACE.extend(face)
        
    RHAND = np.array(RHAND,dtype=np.float64)
    LHAND = np.array(LHAND,dtype=np.float64)
    RPOSE = np.array(RPOSE,dtype=np.float64)
    LPOSE = np.array(LPOSE,dtype=np.float64)
    FACE = np.array(FACE, dtype=np.float64)
    gc.collect()

    RHM = np.mean(RHAND, axis=0)
    LHM = np.mean(LHAND, axis=0)
    RPM = np.mean(RPOSE, axis=0)
    LPM = np.mean(LPOSE, axis=0)
    FACEM = np.mean(FACE, axis=0)

    RHS = np.std(RHAND, axis=0)
    LHS = np.std(LHAND, axis=0)
    RPS = np.std(RPOSE, axis=0)
    LPS = np.std(LPOSE, axis=0)
    FACES = np.std(FACE, axis=0)

    # npyPath = "kaggle/input/aslfr-dataset-tfrecords"
    npyPath = r"C:/Users/chuqi/ac215/kaggle-data/capy_tfds/mean_std"
    fmkdir(os.path.join(npyPath,"mean_std"))
    np.save(os.path.join(npyPath,"mean_std/rh_mean.npy"), RHM)
    np.save(os.path.join(npyPath,"mean_std/lh_mean.npy"), LHM)
    np.save(os.path.join(npyPath,"mean_std/rp_mean.npy"), RPM)
    np.save(os.path.join(npyPath,"mean_std/lp_mean.npy"), LPM)
    np.save(os.path.join(npyPath,"mean_std/face_mean.npy"), FACEM)

    np.save(os.path.join(npyPath,"mean_std/rh_std.npy"), RHS)
    np.save(os.path.join(npyPath,"mean_std/lh_std.npy"), LHS)
    np.save(os.path.join(npyPath,"mean_std/rp_std.npy"), RPS)
    np.save(os.path.join(npyPath,"mean_std/lp_std.npy"), LPS)
    np.save(os.path.join(npyPath,"mean_std/face_std.npy"), FACES)

In [12]:
# FRAME_LEN = 128
FRAME_LEN = 128*2

@tf.function()
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]), constant_values=float("NaN"))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

@tf.function(jit_compile=True)
def pre_process0(x):    
    face_x = tf.gather(x, FACE_IDX_X, axis=1)
    face_y = tf.gather(x, FACE_IDX_Y, axis=1)
    face_z = tf.gather(x, FACE_IDX_Z, axis=1)

    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)
    
    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)
    
    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)
    
    face   = tf.concat([face_x[..., tf.newaxis], face_y[..., tf.newaxis], face_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)
    
    hand = tf.concat([rhand, lhand], axis=1)
    hand = tf.where(tf.math.is_nan(hand), 0.0, hand)
    mask = tf.math.not_equal(tf.reduce_sum(hand, axis=[1, 2]), 0.0)
    

    face = face[mask]
    rhand = rhand[mask]
    lhand = lhand[mask]
    rpose = rpose[mask]
    lpose = lpose[mask]

    return face, rhand, lhand, rpose, lpose

@tf.function()
def pre_process1(face, rhand, lhand, rpose, lpose):
    print(type(face),face.shape)
    face   = (resize_pad(face) - FACEM) / FACES
    rhand = (resize_pad(rhand) - RHM) / RHS
    lhand = (resize_pad(lhand) - LHM) / LHS
    rpose = (resize_pad(rpose) - RPM) / RPS
    lpose = (resize_pad(lpose) - LPM) / LPS

    x = tf.concat([face, rhand, lhand, rpose, lpose], axis=1)
    s = tf.shape(x)
    x = tf.reshape(x, (s[0], s[1]*s[2]))
    x = tf.where(tf.math.is_nan(x), 0.0, x)
    return x

# frames = tf.cast(frames, dtype=tf.float32)
frames_convert = frames.astype(float)
frames_tf = tf.cast(frames_convert, dtype=tf.float32)
pre0 = pre_process0(frames_tf)
pre1 = pre_process1(*pre0)
INPUT_SHAPE = list(pre1.shape)
print(INPUT_SHAPE)

In [13]:
# Build TFrecords
isGenTFrecord = True
if isGenTFrecord:
    fmkdir(r"C:/Users/chuqi/ac215/capy_data_test/test_tfds")
    for file_id in tqdm.tqdm(os.listdir('C:/Users/chuqi/ac215/capy_data_test/test_npy')):
        tffile = f"C:/Users/chuqi/ac215/capy_data_test/test_tfds/{file_id}.tfrecord"
        frames = load_npy_data(file_id)
        phrase = str(df[df['FILE_ID']==file_id]['SENTENCE'].iloc[0])
        frames_convert = frames.astype(float)
        frames_tf = tf.cast(frames_convert, dtype=tf.float32)
        print(frames.shape, phrase)
        
        with tf.io.TFRecordWriter(tffile) as file_writer:
            face, rhand, lhand, rpose, lpose = pre_process0(frames_tf)

            if max(rhand.shape[0], lhand.shape[0]) < len(phrase):
                continue
                    
            features = {}
            features["face"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(face, -1).numpy())) 
            features["rhand"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(rhand, -1).numpy())) 
            features["lhand"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(lhand, -1).numpy())) 
            features["rpose"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(rpose, -1).numpy())) 
            features["lpose"] = tf.train.Feature(float_list=tf.train.FloatList(value=tf.reshape(lpose, -1).numpy())) 
            features["phrase"] = tf.train.Feature(int64_list=tf.train.Int64List(value=[char_to_num[x] for x in phrase]))
                
            record_bytes = tf.train.Example(features=tf.train.Features(feature=features)).SerializeToString()
            file_writer.write(record_bytes)

In [14]:
def decode_fn(record_bytes):
    schema = {
        "face": tf.io.VarLenFeature(tf.float32),
        "rhand": tf.io.VarLenFeature(tf.float32),
        "lhand": tf.io.VarLenFeature(tf.float32),
        "rpose": tf.io.VarLenFeature(tf.float32),
        "lpose": tf.io.VarLenFeature(tf.float32),
        "phrase": tf.io.VarLenFeature(tf.int64)
    }
    x = tf.io.parse_single_example(record_bytes, schema)

    face = tf.reshape(tf.sparse.to_dense(x["face"]), (-1, 40, 3))
    rhand = tf.reshape(tf.sparse.to_dense(x["rhand"]), (-1, 21, 3))
    lhand = tf.reshape(tf.sparse.to_dense(x["lhand"]), (-1, 21, 3))
    rpose = tf.reshape(tf.sparse.to_dense(x["rpose"]), (-1, 5, 3))
    lpose = tf.reshape(tf.sparse.to_dense(x["lpose"]), (-1, 5, 3))
    phrase = tf.sparse.to_dense(x["phrase"])

    return face, rhand, lhand, rpose, lpose, phrase

def pre_process_fn(lip, rhand, lhand, rpose, lpose, phrase):
    phrase = tf.pad(phrase, [[0, MAX_PHRASE_LENGTH-tf.shape(phrase)[0]]], constant_values=pad_token_idx)
    return pre_process1(lip, rhand, lhand, rpose, lpose), phrase

MAX_PHRASE_LENGTH = 500
tffiles = [f"C:/Users/chuqi/ac215/capy_data_test/test_tfds/{file_id}.tfrecord" for file_id in os.listdir('C:/Users/chuqi/ac215/capy_data_test/test_npy')]
val_len = 1#int(0.05 * len(pqfiles))
train_batch_size = 32
val_batch_size = 32

train_dataset =  tf.data.TFRecordDataset(tffiles[val_len:]).prefetch(tf.data.AUTOTUNE).shuffle(5000).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(train_batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset =  tf.data.TFRecordDataset(tffiles[:val_len]).prefetch(tf.data.AUTOTUNE).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(val_batch_size).prefetch(tf.data.AUTOTUNE)

batch = next(iter(val_dataset))
batch[0].shape, batch[1].shape
print(batch[0].shape)
print(batch[1].shape)

In [15]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=6, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

def positional_encoding(maxlen, num_hid):
        depth = num_hid/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = tf.concat(
          [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
          axis=-1)
        return pos_encoding
def positional_encoding2(maxlen, num_hid):
        depth = num_hid/2
        positions = tf.range(maxlen, dtype = tf.float32)[..., tf.newaxis]
        depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
        angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
        angle_rads = tf.linalg.matmul(positions, angle_rates)
        pos_encoding = np.zeros((maxlen, num_hid))
        pos_encoding[:, 0::2] = np.sin(angle_rads)
        pos_encoding[:, 1::2] = np.cos(angle_rads)
        return pos_encoding

In [16]:
def CTCLoss(labels, logits):
    label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
    logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    loss = tf.nn.ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
            logits_time_major=False
        )
    loss = tf.reduce_mean(loss)
    return loss

In [17]:
def get_model(dim = 384):
    inp = tf.keras.Input(INPUT_SHAPE)
    x = tf.keras.layers.Masking(mask_value=0.0)(inp)
    x = tf.keras.layers.Dense(dim, use_bias=False, name='stem_conv')(x)
    pe = tf.cast(positional_encoding(INPUT_SHAPE[0], dim), dtype=x.dtype)
    x = x + pe
    x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)
    num_blocks = 6
    drop_rate  = 0.4
    for i in range(num_blocks):
        x = Conv1DBlock(dim, 11, drop_rate=drop_rate)(x)
        x = Conv1DBlock(dim,  5, drop_rate=drop_rate)(x)
        x = Conv1DBlock(dim,  3, drop_rate=drop_rate)(x)
        x = TransformerBlock(dim, expand=2)(x)

    x = tf.keras.layers.Dense(dim*2,activation='relu',name='top_conv')(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    #x = LateDropout(0.7)(x)
    x = tf.keras.layers.Dense(len(char_to_num),name='classifier')(x)

    model = tf.keras.Model(inp, x)

    loss = CTCLoss
    
    # Adam Optimizer
    optimizer = tfa.optimizers.RectifiedAdam(sma_threshold=4)
    optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)

    model.compile(loss=loss, optimizer=optimizer)

    return model

tf.keras.backend.clear_session()
model = get_model()
model(batch[0])
model.summary()

In [18]:
def num_to_char_fn(y):
    return [num_to_char.get(x, "") for x in y]

@tf.function()
def decode_phrase(pred):
    x = tf.argmax(pred, axis=1)
    diff = tf.not_equal(x[:-1], x[1:])
    adjacent_indices = tf.where(diff)[:, 0]
    x = tf.gather(x, adjacent_indices)
    mask = x != pad_token_idx
    x = tf.boolean_mask(x, mask, axis=0)
    return x

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    output_text = []
    for result in pred:
        result = "".join(num_to_char_fn(decode_phrase(result).numpy()))
        output_text.append(result)
    return output_text

In [19]:
# A callback class to output a few transcriptions during training
class CallbackEval(tf.keras.callbacks.Callback):
    """Displays a batch of outputs after every epoch."""

    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset

    def on_epoch_end(self, epoch: int, logs=None):
        model.save_weights("model.h5")
        predictions = []
        targets = []
        for batch in self.dataset:
            X, y = batch
            batch_predictions = model(X)
            batch_predictions = decode_batch_predictions(batch_predictions)
            predictions.extend(batch_predictions)
            for label in y:
                label = "".join(num_to_char_fn(label.numpy()))
                targets.append(label)
        print("-" * 100)
        # for i in np.random.randint(0, len(predictions), 2):
        print(f"Target    : {targets}")
        print(f"Prediction: {predictions}, len: {len(predictions)}")
        print("-" * 100)
        
#         for i in range(32):
#             print(f"Target    : {targets[i]}")
#             print(f"Prediction: {predictions[i]}, len: {len(predictions[i])}")
#             print("-" * 100)

# Callback function to check transcription on the val set.
validation_callback = CallbackEval(val_dataset.take(1))


In [20]:
N_EPOCHS = 51
N_WARMUP_EPOCHS = 10
LR_MAX = 1e-3
WD_RATIO = 0.05
WARMUP_METHOD = "exp"

In [21]:

def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=N_EPOCHS):
    
    if current_step < num_warmup_steps:
        if WARMUP_METHOD == 'log':
            return lr_max * 0.10 ** (num_warmup_steps - current_step)
        else:
            return lr_max * 2 ** -(num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max
    
def plot_lr_schedule(lr_schedule, epochs):
    fig = plt.figure(figsize=(20, 10))
    plt.plot([None] + lr_schedule + [None])
    # X Labels
    x = np.arange(1, epochs + 1)
    x_axis_labels = [i if epochs <= 40 or i % 5 == 0 or i == 1 else None for i in range(1, epochs + 1)]
    plt.xlim([1, epochs])
    plt.xticks(x, x_axis_labels) # set tick step to 1 and let x axis start at 1
    
    # Increase y-limit for better readability
    plt.ylim([0, max(lr_schedule) * 1.1])
    
    # Title
    schedule_info = f'start: {lr_schedule[0]:.1E}, max: {max(lr_schedule):.1E}, final: {lr_schedule[-1]:.1E}'
    plt.title(f'Step Learning Rate Schedule, {schedule_info}', size=18, pad=12)
    
    # Plot Learning Rates
    for x, val in enumerate(lr_schedule):
        if epochs <= 40 or x % 5 == 0 or x is epochs - 1:
            if x < len(lr_schedule) - 1:
                if lr_schedule[x - 1] < val:
                    ha = 'right'
                else:
                    ha = 'left'
            elif x == 0:
                ha = 'right'
            else:
                ha = 'left'
            plt.plot(x + 1, val, 'o', color='black');
            offset_y = (max(lr_schedule) - min(lr_schedule)) * 0.02
            plt.annotate(f'{val:.1E}', xy=(x + 1, val + offset_y), size=12, ha=ha)
    
    plt.xlabel('Epoch', size=16, labelpad=5)
    plt.ylabel('Learning Rate', size=16, labelpad=5)
    plt.grid()
    plt.show()

# Learning rate for encoder
LR_SCHEDULE = [lrfn(step, num_warmup_steps=N_WARMUP_EPOCHS, lr_max=LR_MAX, num_cycles=0.50) for step in range(N_EPOCHS)]
# Plot Learning Rate Schedule
plot_lr_schedule(LR_SCHEDULE, epochs=N_EPOCHS)
# Learning Rate Callback
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step], verbose=0)

# Custom callback to update weight decay with learning rate
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='code/baseline2/out2/model.{epoch:05d}.keras',
    save_weights_only=True,
    period=50)

In [22]:
model.load_weights(r"C:/Users/chuqi/ac215/kaggle-data/model.h5")

In [23]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
#     epochs=N_EPOCHS,
    epochs=1,
    callbacks=[
        checkpoint_callback,
        validation_callback,
        lr_callback,
        WeightDecayCallback(),
    ]
)

In [ ]:
from wandb.keras import WandbCallback, WandbMetricsLogger
# Initialize a W&B run
wandb.init(
    project = 'capy-train',
    config = {
      "learning_rate": LR_MAX,
      "epochs": 1,
      "batch_size": 32,
      "model_name": model.name
    },
    name = model.name
)

# Train model
# start_time = time.time()
training_results = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=1,
        callbacks=[WandbCallback()],
        #callbacks = [WandbMetricsLogger(log_freq=1)],
        verbose=1)
# execution_time = (time.time() - start_time)/60.0
# print("Training execution time (mins)",execution_time)

# Update W&B
# wandb.config.update({"execution_time": execution_time})
# Close the W&B run
wandb.run.finish()